# **ИССЛЕДОВАНИЕ ДАННЫХ HR-АГЕНТСТВА**

#### HR-агентство изучает тренды на рынке труда в IT. Компания хочет провести исследование на основе данных о зарплатах в сфере Data Science за 2020–2022 годы и получить некоторые выводы.

* ``work_year`` - Год, в котором была выплачена зарплата.
* ``experience_level`` - Опыт работы на этой должности в течение года.
* ``employment_type`` - Тип трудоустройства для этой роли.
* ``job_title`` - Роль, в которой соискатель работал в течение года.
* ``salary`` - Общая выплаченная валовая сумма заработной платы.
* ``salary_currency`` - Валюта выплачиваемой заработной платы в виде кода валюты ISO 4217.
* ``salary_in_usd`` - Зарплата в долларах США (валютный курс, делённый на среднее значение курса доллара США за соответствующий год через fxdata.foorilla.com).
* ``employee_residence`` - Основная страна проживания сотрудника в течение рабочего года в виде кода страны ISO 3166.
* ``remote_ratio`` - Общий объём работы, выполняемой удалённо. 
* ``company_location`` - Страна главного офиса работодателя или филиала по контракту в виде кода страны ISO 3166.
* ``company_size`` - Среднее количество людей, работавших в компании в течение года.

Исследуйте данные и сделайте выводы по полученным результатам. Подкрепите свои рассуждения и выводы визуализациями и с помощью статистического тестирования проверьте, являются ли выводы статистически значимыми.

В процессе своего анализа вы должны:
1. Выяснить, какие факторы влияют на зарплату специалиста Data Scientist.
2. А также ответить на ключевые вопросы HR-агентства:
* Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?
* Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?
* Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?
* Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?

Если вы найдёте в данных интересные закономерности, также отметьте их в своём анализе.

Продемонстрируйте использование разных тестов для проверки статистической значимости сделанных выводов:

* тесты для количественного признака:
* * для одной выборки;
* * для двух выборок;
* * для нескольких выборок;

* тест для категориальных признаков.

In [83]:
import pandas as pd
from IPython.display import Markdown
import plotly.express as px
import plotly.io as pio
import numpy as np

# pio.renderers.default = 'png'
pio.templates.default = 'plotly_dark'

## **Загрузка и обработка данных**

In [84]:
# Знакомсво с данными
df = pd.read_csv('data/ds_salaries.csv', index_col=0)
display(Markdown('### Просматриваем набор данных')),
display(df.sample(5)),
display(Markdown('### Характеристики категориальных столбцов')),
display(df.describe(include=object).T),
display(Markdown('### Характеристики числовых столбцов')),
display(df.describe().T)
display(Markdown('### Характеристики столбцов')),
df.info(),
print('\n')
print('Количество дубликатов:', df.duplicated().sum(), '\n')
print('Количество пропусков:', df.isna().sum().sum(), '\n')
nums_list_col = [col for col in df.columns if df[col].dtype != object]
cat_list_col = [col for col in df.columns if df[col].dtype == object]
print(f'Количество числовых столбцов: {len(nums_list_col)}\n{nums_list_col}\n')
print(f'Количество категориальных столбцов: {len(cat_list_col)}\n{cat_list_col}')

### Просматриваем набор данных

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
378,2022,SE,FT,Data Architect,208775,USD,208775,US,100,US,M
315,2022,SE,FT,Data Scientist,165220,USD,165220,US,100,US,M
328,2022,SE,FT,Data Analyst,170000,USD,170000,US,100,US,M
129,2021,SE,FT,Lead Data Scientist,3000000,INR,40570,IN,50,IN,L
19,2020,MI,FT,Lead Data Engineer,56000,USD,56000,PT,100,US,M


### Характеристики категориальных столбцов

,count,unique,top,freq
experience_level,607,4,SE,280
employment_type,607,4,FT,588
job_title,607,50,Data Scientist,143
salary_currency,607,17,USD,398
employee_residence,607,57,US,332
company_location,607,50,US,355
company_size,607,3,M,326


### Характеристики числовых столбцов

,count,mean,std,min,25%,50%,75%,max
work_year,607.0,2021.405272,6.921330e-01,2020.0,2021.0,2022.0,2022.0,2022.0
salary,607.0,324000.062603,1.544357e+06,4000.0,70000.0,115000.0,165000.0,30400000.0
salary_in_usd,607.0,112297.869852,7.095726e+04,2859.0,62726.0,101570.0,150000.0,600000.0
remote_ratio,607.0,70.922570,4.070913e+01,0.0,50.0,100.0,100.0,100.0


### Характеристики столбцов

<class 'pandas.core.frame.DataFrame'>
Index: 607 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           607 non-null    int64 
 1   experience_level    607 non-null    object
 2   employment_type     607 non-null    object
 3   job_title           607 non-null    object
 4   salary              607 non-null    int64 
 5   salary_currency     607 non-null    object
 6   salary_in_usd       607 non-null    int64 
 7   employee_residence  607 non-null    object
 8   remote_ratio        607 non-null    int64 
 9   company_location    607 non-null    object
 10  company_size        607 non-null    object
dtypes: int64(4), object(7)
memory usage: 56.9+ KB


Количество дубликатов: 42 

Количество пропусков: 0 

Количество числовых столбцов: 4
['work_year', 'salary', 'salary_in_usd', 'remote_ratio']

Количество категориальных столбцов: 7
['experience_level', 'employment_type', '

### Удаляем дубликаты

In [85]:
df.drop_duplicates(inplace=True)
print('Количество дубликатов:', df.duplicated().sum())

Количество дубликатов: 0


### Удаляем не информативные признаки

К неинформативным признакам относятся ``salary``, ``salary_currency`` так как у нас уже есть признак ``salary_in_usd``, который является общим и где все зарплаты приведены к единому масштабу **USD**.

In [86]:
df.drop(columns=['salary', 'salary_currency'], inplace=True)

## Разведывательный анализ данных




### Визуальный анализ всех данных

In [89]:
fig = px.histogram(
    df,
    x='salary_in_usd',
    title='Распределение зарплат в датасете',
    width=1000,
    height=500,
    color_discrete_sequence=['#ff246e'],
    text_auto=True,
    marginal='box',
    labels={'salary_in_usd': 'ЗП USD'},
)
fig

По графику видно, что распределение зарплат близко к нормальному. 

Прологарифмируем признак, что бы привести его к нормальному распределению.